In [ ]:
input_dir = '/home/matteo/ctasoft/resources'
gamma_diffuse = 'gamma_20deg_0deg_run101___cta-prod3-lapalma3-2147m-LaPalma.simtel.gz'
sys_path = '/home/matteo/ctasoft/'
prepared_data_dir = 'prepared_data/output'

In [ ]:
import sys
import os
sys.path.append(sys_path + "cta-benchmarks/Preparation/parameters_search")  # path contains python_file.py
from quality_parameters_function import iterate_param, create_possible_parameters

In [ ]:
"""
script to find the optimized levels of cleaning for each image of an event.

Data are saved in the script's path in a .npz file

"""

from ctapipe.utils.datasets import find_in_path
from ctapipe.io import event_source
from ctapipe.calib import CameraCalibrator
import numpy as np

possible_cleaning_levels = create_possible_parameters()

quality_param_list = []
hillas_containers = {}

data = []

In [ ]:
prepared_data_dir = '../../../' + prepared_data_dir

if not os.path.exists(prepared_data_dir):
    os.makedirs(prepared_data_dir, exist_ok=True)

In [ ]:
input_url = find_in_path(gamma_diffuse, input_dir)
events = event_source(input_url, max_events=1)

calibrator = CameraCalibrator()

In [ ]:
for number_event, event in enumerate(events):
    hillas_containers.clear()
    calibrator(event)
    data.append((number_event, event.mc.energy, event.inst.subarray.tel_coords))

    for telescope_id, dl1 in event.dl1.tel.items():
        quality_param_list.clear()

        image = dl1.image
        camera = event.inst.subarray.tels[telescope_id].camera

        optimized_cleaning_level = iterate_param(possible_cleaning_levels, event, camera, image,
                                                 telescope_id, quality_param_list, hillas_containers)

        data.append((camera.cam_id, telescope_id,
                     optimized_cleaning_level))

np.savez_compressed(prepared_data_dir+'/parameters_' + gamma_diffuse + '.npz', data=data)

loaded = np.load(prepared_data_dir+'/parameters_' + gamma_diffuse + '.npz', allow_pickle=True)

data_loaded = loaded['data']

print(data_loaded)